## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_0 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o0/o0_bincore8.csv", index_col=0)
print(bin8_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_0.reset_index(inplace=True, drop=True)

print('shape -', bin8_0.shape)
print('reset_index 완료')
print('input data shape')
bin8_0.head()

(30111221, 2)
shape - (30111221, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_0['label'].value_counts())

256
0    30060985
1       50236
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_0[bin8_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_0', len(ls_idx_bin))

# loc 로 수정필요
bin8_0_Ngram = bin8_0.loc[ls_idx_bin,:].copy()

803776
나머지 0
최종 길이 803776
bin8_0 803776


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

79730.0
0 27893635
1000 22113236
2000 22810738
3000 29763201
4000 13487996
5000 20028829
6000 23212675
7000 10043374
8000 1870256
9000 7717973
10000 248140
11000 25057245
12000 29208168
13000 13010507
14000 9084269
15000 20766201
16000 16732068
17000 561510
18000 23443829
19000 14691836
20000 25933862
21000 21792948
22000 7551461
23000 8096856
24000 23204085
25000 24389155
26000 6134588
27000 3012152
28000 19493623
29000 23129798
30000 25264001
31000 23852215
32000 23231736
33000 9529692
34000 11940425
35000 3242907
36000 2100906
37000 15561180
38000 4707753
39000 19776925
40000 28147135
41000 365159
42000 850681
43000 23540147
44000 27186345
45000 3792810
46000 5806374
47000 27093561
48000 18148488
49000 29802770
50000 12159098
51000 26181526
52000 5023832
53000 4852134
54000 16715359
55000 29502015
56000 11485799
57000 25471396
58000 13928870
59000 17418161
60000 27964220
61000 12764441
62000 12586889
63000 12878814
64000 13411087
65000 10944994
66000 21461672
67000 25200687
68000 14

In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_0['label'].value_counts()[1])

79730
50236


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_0_Ngram])
final.shape

(2551360, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_0_onehot_Ngram = pd.concat([final['label'], bc8_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_0_onehot_Ngram.shape)

원핫인코딩완료
(2551360, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_0 = bc8_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_0 = bc8_0_onehot_Ngram['label'].to_numpy()
print(x_bc8_0.shape, x_bc8_0.shape)

x_bc8_0 = x_bc8_0.reshape(-1, right_idx, x_bc8_0.shape[1])
y_bc8_0 = y_bc8_0.reshape(-1, right_idx, 1)

print(x_bc8_0.shape, y_bc8_0.shape)

(2551360, 256) (2551360, 256)
(159460, 16, 256) (159460, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_0.shape[0])

x_bc8_0 = x_bc8_0[p]
y_bc8_0 = y_bc8_0[p]

print(x_bc8_0.shape, y_bc8_0.shape)

(159460, 16, 256) (159460, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_0, y_bc8_0):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_0[train],
               y_bc8_0[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_0[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_0[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 103s 718us/sample - loss: 0.0078 - accuracy: 0.9982
Epoch 2/10
143514/143514 [==============================] - 99s 693us/sample - loss: 8.3870e-04 - accuracy: 0.9998
Epoch 3/10
143514/143514 [==============================] - 108s 750us/sample - loss: 6.0297e-04 - accuracy: 

accuracy_score 0.999862818261633
recall_score 0.9970474257243034
precision_score 0.9964957580228698
f1_score 0.9967715155428466
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 132s 918us/sample - loss: 3.1761e-04 - accuracy: 0.9999
Epoch 2/10
143514/143514 [==============================] - 109s 762us/sample - loss: 2.89

accuracy_score 0.9998667377398721
recall_score 0.9959388899632566
precision_score 0.9974820840596552
f1_score 0.9967098896845364
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 107s 744us/sample - loss: 2.7535e-04 - accuracy: 0.9999
Epoch 2/10
143514/143514 [==============================] - 87s 606us/sample - loss: 2.54

accuracy_score 0.9999490467828923
recall_score 0.9988704819277109
precision_score 0.9986824769433466
f1_score 0.9987764705882353
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 134s 930us/sample - loss: 1.9544e-04 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 104s 727us/sample - loss: 1.8

accuracy_score 0.999933368869936
recall_score 0.99797197640118
precision_score 0.9988927846466138
f1_score 0.9984321682191276
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 99s 689us/sample - loss: 1.7941e-04 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 80s 561us/sample - loss: 1.4185e-

...

accuracy_score 0.9999804026088047
recall_score 0.9992363497518136
precision_score 0.9998089780324737
f1_score 0.999522581877208
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 102s 711us/sample - loss: 8.7899e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 88s 612us/sample - loss: 7.185

accuracy_score 0.9999804026088047
recall_score 0.9998122418325197
precision_score 0.9992493901294802
f1_score 0.9995307367433132
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 105s 731us/sample - loss: 6.0306e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 92s 641us/sample - loss: 3.96

accuracy_score 0.9999843220870438
recall_score 1.0
precision_score 0.9992445703493862
f1_score 0.9996221424522955
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 52s 360us/sample - loss: 3.6437e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 45s 313us/sample - loss: 4.0936e-05 - accurac

accuracy_score 0.9999843220870438
recall_score 1.0
precision_score 0.9992418498862775
f1_score 0.9996207811907472
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 51s 356us/sample - loss: 3.7827e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 44s 305us/sample - loss: 3.4818e-05 - accurac

accuracy_score 0.9999960805217609
recall_score 1.0
precision_score 0.9998121360135263
f1_score 0.999906059182715

K-fold cross validation Accuracy: [0.999862818261633, 0.9998667377398721, 0.9999490467828923, 0.999933368869936, 0.9999647246958485, 0.9999804026088047, 0.9999804026088047, 0.9999843220870438, 0.9999843220870438, 0.9999960805217609]

K-fold cross validation Recall: [0.9970474257243034, 0.9959388899632566, 0.9988704819277109, 0.99797197640118, 0.9998130841121495, 0.9992363497518136, 0.9998122418325197, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9964957580228698, 0.9974820840596552, 0.9986824769433466, 0.9988927846466138, 0.9985066268433825, 0.9998089780324737, 0.9992493901294802, 0.9992445703493862, 0.9992418498862775, 0.9998121360135263]

K-fold cross validation F1-Score: [0.9967715155428466, 0.9967098896845364, 0.9987764705882353, 0.9984321682191276, 0.9991594284113197, 0.999522581877208, 0.9995307367433132, 0.9996221424522955, 0.9996207811907472, 0.999906059182

## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.999950222626364
10-Fold Cross_validation. Recall : 0.9988690449712934
10-Fold Cross_validation. Precision : 0.9987416654927012
10-Fold Cross_validation. F1-Score : 0.9988051773892345


In [13]:
model1.save('gcc8_bin_core_s16_h24_o0.h5')
print('save 완료')

save 완료
